In [1]:
import awkward as ak
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np

from helpers.physics_functions import *

In [2]:
muon_vars = ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge", "Muon_pfRelIso03_all", "Muon_pfRelIso04_all"]
jet_vars = ["Jet_pt", "Jet_eta", "Jet_phi", "Jet_mass", "Jet_nConstituents", "Jet_btagCSVV2", "Jet_btagDeepB", "Jet_btagDeepFlavB", "MET_pt", "MET_sumEt"]

path_to_input = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/skimmed_data_2016H_30555/"
path_to_output = f"/pscratch/sd/r/rmastand/dimuonAD/compiled_data/eta/"

num_files = 28


In [3]:


all_data = {
    "muon_pt": [],
      "amuon_pt": [],
      "muon_eta": [],
      "amuon_eta": [],
      "muon_iso03": [],
      "amuon_iso03": [],
      "muon_iso04": [],
      "amuon_iso04": [],
      "jet_pt": [],
      "jet_eta": [],
      "jet_mass": [],
      "dimu_pt": [],
      "dimu_eta": [],
      "dimu_mass": [],
    "dimu_mass_samesign": [],
    }


In [4]:
for i in range(num_files):
    
    print(f"Analyzing file {i+1} of {num_files}...")
    
    # LOAD IN DATA
    
    with open(f"{path_to_input}/all_mu_{i}", "rb") as input_file:
        loc_mu_feature = pickle.load(input_file)
        
    with open(f"{path_to_input}/all_jet_{i}", "rb") as input_file:
        loc_jet_feature = pickle.load(input_file)
        

    # START ANALYSIS
    
    loc_muon_filter = loc_mu_feature["Muon_charge"] == -1
    loc_amuon_filter = loc_mu_feature["Muon_charge"] == 1
    
    """
    EVENTS WITH 1 MU, 1 AMU
    """
    # First get valid events, i.e. events with 1 muon, 1 amuon
    
    # get mp system
    
    mp_event_filter = (np.sum(loc_muon_filter, axis = 1) > 0) & (np.sum(loc_amuon_filter, axis = 1) > 0)
    passed_mu_pt = loc_mu_feature["Muon_pt"][loc_muon_filter][mp_event_filter][:,0]
    passed_amu_pt = loc_mu_feature["Muon_pt"][loc_amuon_filter][mp_event_filter][:,0] 
    passed_mu_eta = loc_mu_feature["Muon_eta"][loc_muon_filter][mp_event_filter][:,0]
    passed_amu_eta = loc_mu_feature["Muon_eta"][loc_amuon_filter][mp_event_filter][:,0]
    passed_mu_phi = loc_mu_feature["Muon_phi"][loc_muon_filter][mp_event_filter][:,0]
    passed_amu_phi = loc_mu_feature["Muon_phi"][loc_amuon_filter][mp_event_filter][:,0]
        
    dimu_mass, dimu_pt, dimu_eta, _ = assemble_m_inv(muon_mass, passed_mu_pt, passed_mu_eta, passed_mu_phi, 
                                   muon_mass, passed_amu_pt,  passed_amu_eta,  passed_amu_phi)

    all_data["dimu_mass"].append(dimu_mass.to_numpy(allow_missing = True))
    all_data["dimu_pt"].append(dimu_pt.to_numpy(allow_missing = True))
    all_data["dimu_eta"].append(dimu_eta.to_numpy(allow_missing = True))
    
    all_data["muon_iso04"].append(ak.firsts(loc_mu_feature["Muon_pfRelIso04_all"][loc_muon_filter][mp_event_filter]).to_numpy(allow_missing = True))
    all_data["amuon_iso04"].append(ak.firsts(loc_mu_feature["Muon_pfRelIso04_all"][loc_amuon_filter][mp_event_filter]).to_numpy(allow_missing = True))
  
    all_data["muon_iso03"].append(ak.firsts(loc_mu_feature["Muon_pfRelIso03_all"][loc_muon_filter][mp_event_filter]).to_numpy(allow_missing = True))
    all_data["amuon_iso03"].append(ak.firsts(loc_mu_feature["Muon_pfRelIso03_all"][loc_amuon_filter][mp_event_filter]).to_numpy(allow_missing = True))
  
    all_data["muon_pt"].append(ak.firsts(loc_mu_feature["Muon_pt"][loc_muon_filter][mp_event_filter]).to_numpy(allow_missing = True))
    all_data["amuon_pt"].append(ak.firsts(loc_mu_feature["Muon_pt"][loc_amuon_filter][mp_event_filter]).to_numpy(allow_missing = True))
    all_data["muon_eta"].append(ak.firsts(loc_mu_feature["Muon_eta"][loc_muon_filter][mp_event_filter]).to_numpy(allow_missing = True))
    all_data["amuon_eta"].append(ak.firsts(loc_mu_feature["Muon_eta"][loc_amuon_filter][mp_event_filter]).to_numpy(allow_missing = True))
  
    all_data["jet_pt"].append(ak.firsts(loc_jet_feature["Jet_pt"][mp_event_filter]).to_numpy(allow_missing = True))
    all_data["jet_eta"].append(ak.firsts(loc_jet_feature["Jet_eta"][mp_event_filter]).to_numpy(allow_missing = True))
    all_data["jet_mass"].append(ak.firsts(loc_jet_feature["Jet_mass"][mp_event_filter]).to_numpy(allow_missing = True))

    
    """
    SAME-SIGN MUONS FOR BACKGROUND ESTIMATION
    """
    
    
    
    # get mm system
    
    mm_event_filter = (np.sum(loc_muon_filter, axis = 1) > 1)
    passed_mu_pt = loc_mu_feature["Muon_pt"][loc_muon_filter][mm_event_filter][:,0]
    passed_amu_pt = loc_mu_feature["Muon_pt"][loc_muon_filter][mm_event_filter][:,1] 
    passed_mu_eta = loc_mu_feature["Muon_eta"][loc_muon_filter][mm_event_filter][:,0]
    passed_amu_eta = loc_mu_feature["Muon_eta"][loc_muon_filter][mm_event_filter][:,1]
    passed_mu_phi = loc_mu_feature["Muon_phi"][loc_muon_filter][mm_event_filter][:,0]
    passed_amu_phi = loc_mu_feature["Muon_phi"][loc_muon_filter][mm_event_filter][:,1]
        
    dimu_mass, _, _, _ = assemble_m_inv(muon_mass, passed_mu_pt, passed_mu_eta, passed_mu_phi, 
                                   muon_mass, passed_amu_pt,  passed_amu_eta,  passed_amu_phi)

    all_data["dimu_mass_samesign"].append(dimu_mass.to_numpy(allow_missing = True))
    
    # get pp system
    
    pp_event_filter = (np.sum(loc_amuon_filter, axis = 1) > 1)
    passed_mu_pt = loc_mu_feature["Muon_pt"][loc_amuon_filter][pp_event_filter][:,0]
    passed_amu_pt = loc_mu_feature["Muon_pt"][loc_amuon_filter][pp_event_filter][:,1] 
    passed_mu_eta = loc_mu_feature["Muon_eta"][loc_amuon_filter][pp_event_filter][:,0]
    passed_amu_eta = loc_mu_feature["Muon_eta"][loc_amuon_filter][pp_event_filter][:,1]
    passed_mu_phi = loc_mu_feature["Muon_phi"][loc_amuon_filter][pp_event_filter][:,0]
    passed_amu_phi = loc_mu_feature["Muon_phi"][loc_amuon_filter][pp_event_filter][:,1]
        
    dimu_mass, _, _, _ = assemble_m_inv(muon_mass, passed_mu_pt, passed_mu_eta, passed_mu_phi, 
                                   muon_mass, passed_amu_pt,  passed_amu_eta,  passed_amu_phi)

    all_data["dimu_mass_samesign"].append(dimu_mass.to_numpy(allow_missing = True))


print("Done!")
    
    

Analyzing file 0 of 28...


/global/homes/r/rmastand/.conda/envs/cernroot/lib/python3.11/site-packages/awkward/_nplikes/array_module.py:245: RuntimeWarning: invalid value encountered in sqrt
  return impl(*broadcasted_args, **(kwargs or {}))


Analyzing file 1 of 28...
Analyzing file 2 of 28...
Analyzing file 3 of 28...
Analyzing file 4 of 28...
Analyzing file 5 of 28...
Analyzing file 6 of 28...
Analyzing file 7 of 28...
Analyzing file 8 of 28...
Analyzing file 9 of 28...
Analyzing file 10 of 28...
Analyzing file 11 of 28...
Analyzing file 12 of 28...
Analyzing file 13 of 28...


/global/homes/r/rmastand/.conda/envs/cernroot/lib/python3.11/site-packages/awkward/_nplikes/array_module.py:245: RuntimeWarning: divide by zero encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))


Analyzing file 14 of 28...
Analyzing file 15 of 28...
Analyzing file 16 of 28...
Analyzing file 17 of 28...
Analyzing file 18 of 28...
Analyzing file 19 of 28...
Analyzing file 20 of 28...
Analyzing file 21 of 28...
Analyzing file 22 of 28...
Analyzing file 23 of 28...
Analyzing file 24 of 28...
Analyzing file 25 of 28...
Analyzing file 26 of 28...
Analyzing file 27 of 28...
Done!


In [5]:
for key in all_data.keys():
    all_data[key] = np.hstack(all_data[key])
    print(key, all_data[key].shape)

    

muon_pt (30701271,)
amuon_pt (30701271,)
muon_eta (30701271,)
amuon_eta (30701271,)
muon_iso03 (30701271,)
amuon_iso03 (30701271,)
muon_iso04 (30701271,)
amuon_iso04 (30701271,)
jet_pt (30701271,)
jet_eta (30701271,)
jet_mass (30701271,)
dimu_pt (30701271,)
dimu_eta (30701271,)
dimu_mass (30701271,)
dimu_mass_samesign (22822590,)


In [6]:
with open(f"{path_to_output}/DATA", "wb") as output_file:
        pickle.dump(all_data, output_file)